In [1]:
import numpy as np
import pandas as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

MIN_MATCH_COUNT = 19
import cv2
import time
import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


In [2]:
#завантажити нейромережу
model = load_model('ReNet')
IMG_SIZE = 128

In [32]:
#завантажити вхідні дані
X = np.zeros((363, IMG_SIZE, IMG_SIZE, 3))
Y = np.zeros((363, 4))
for i in range(121):
    img = cv2.imread(f"img/{i}.jpg")
    x = cv2.resize(img, (128, 128), interpolation = cv2.INTER_AREA)
    x = np.expand_dims(x, axis=0)
    x = x/255.0
    X[i] = x
    if(i <= 102):
        Y[i][0] = 1
    else:
        Y[i][3] = 1
for i in range(121):
    img = cv2.imread(f"img2/{i}.jpg")
    x = cv2.resize(img, (128, 128), interpolation = cv2.INTER_AREA)
    x = np.expand_dims(x, axis=0)
    x = x/255.0
    X[i+121] = x
    if(i <= 100):
        Y[i+121][1] = 1
    else:
        Y[i+121][3] = 1
for i in range(121):
    img = cv2.imread(f"img3/{i}.jpg")
    x = cv2.resize(img, (128, 128), interpolation = cv2.INTER_AREA)
    x = np.expand_dims(x, axis=0)
    x = x/255.0
    X[i+242] = x
    if(i <= 94):
        Y[i+242][2] = 1
    else:
        Y[i+242][3] = 1        

        
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0, test_size=0.2)

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))


number of training examples = 290
number of test examples = 73
X_train shape: (290, 128, 128, 3)
Y_train shape: (290, 4)
X_test shape: (73, 128, 128, 3)
Y_test shape: (73, 4)


In [33]:
#пройти нейромережею по вхідних даних та зібрати метрики
t1 = 0
t2 = 0
t3 = 0
t4 = 0
a1 = 0
a2 = 0
a3 = 0
a4 = 0
times = 0
for i in range(X_test.shape[0]):
    start_time = time.time()
    p = model.predict(np.expand_dims(X_test[i], axis=0))
    times += (time.time() - start_time)
    #for j in range(4):
        #if p[j] == np.max(p):
    yp = np.argmax(p)
    yt = np.argmax(Y_test[i])
    #print(f"{i+1}: {yp} {yt}")
    if yp == yt == 0: t1 += 1
    elif yp == yt == 1: t2 += 1
    elif yp == yt == 2: t3 += 1
    elif yp == yt == 3: t4 += 1
        
    if yp == 0: a1 += 1
    elif yp == 1: a2 += 1
    elif yp == 2: a3 += 1
    elif yp == 3: a4 += 1

sY = np.sum(Y_test, axis = 0)
print(f"detected car:\t\t{t1/sY[0]}")
print(f"detected stand:\t\t{t2/sY[1]}")
print(f"detected dosimeter:\t{t3/sY[2]}")
print(f"nothing detected:\t{t4/sY[3]}\n")

print(f"false positive car:\t{(a1 - t1)/a1}")
print(f"false positive stand:\t{(a2 - t2)/a2}")
print(f"false positive dosimeter:{(a3 - t3)/a3}")
print(f"false negative:\t\t{(a4 - t4)/a4}\n")

print(f"mean time:\t\t{times/X.shape[0]}s")

detected car:		0.8333333333333334
detected stand:		0.8
detected dosimeter:	0.5882352941176471
nothing detected:	1.0

false positive car:	0.047619047619047616
false positive stand:	0.0
false positive dosimeter:0.09090909090909091
false negative:		0.52

mean time:		0.008813564442406016s


In [30]:
#пройти нейромережею та дескриптором по відео, та записати відео з результатами

cap = cv2.VideoCapture("carvideo.mp4")
img1 = cv2.imread("img/0.jpg")

surf = cv2.xfeatures2d.SURF_create()
kp1, des1 = surf.detectAndCompute(img1, None)
#out = cv2.VideoWriter('classificator1.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 1, (1080, 1920))
out = cv2.VideoWriter('classificator.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 20, (960, 540))
while(1):    
    ret, frame = cap.read()
    if ret == True:
        frame = cv2.resize(frame, (0,0), fx=0.5, fy=0.5)
        x = cv2.resize(frame, (128, 128), interpolation = cv2.INTER_AREA)
        x = np.expand_dims(x, axis=0)
        x = x/255.0
        kp2, des2 = surf.detectAndCompute(frame, None)
        bf = cv2.BFMatcher()

        good = []
        if isinstance(des2, np.ndarray):
            matches = bf.knnMatch(des1, des2, k=2)
            for m,n in matches:
                if m.distance < 0.75*n.distance:
                    good.append(m)

        if (len(good)>=MIN_MATCH_COUNT):
            src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
            dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

            M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
            matchesMask = mask.ravel().tolist()

            h,w,c = img1.shape
            pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
            dst = cv2.perspectiveTransform(pts,M)

            frame = cv2.polylines(frame,[np.int32(dst)],True,255,3, cv2.LINE_AA)
        p = model.predict(x)
        obj = ("car","stand","dozimeter","none")
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, obj[np.argmax(p)], (10,50), font, 2, (0, 255, 0), 2, cv2.LINE_AA)
        out.write(frame)
        cv2.imshow("result", frame)
    else:
        break

    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

out.release()    
cap.release()
cv2.destroyAllWindows()

<VideoWriter 000001B99D952450>


In [19]:
shape

(540, 960, 3)

In [ ]:
results_SURF_stand = pd.read_csv('csv/SURF_VladB_dataresult.csv', sep=',')
results_SURF_dosimeter = pd.read_csv('csv/SURF_VladK_dataresult.csv', sep=',')
results_SURF_car = pd.read_csv('csv/SURF_Egor_dataresult.csv', sep=',')

stand_positive = 100
stand_negative = 20

dosimeter_positive = 94
dosimeter_negative = 20

car_positive = 102
car_negative = 18

TP_SURF_stand = results_SURF_stand["MeanDist"].head(stand_positive).count().sum()
TN_SURF_stand = results_SURF_stand["MeanDist"].tail(stand_negative).isna().sum()
FN_SURF_stand = results_SURF_stand["MeanDist"].head(stand_positive).isna().sum()
FP_SURF_stand = results_SURF_stand["MeanDist"].tail(stand_negative).count().sum()
print("SURF:", round(TP_SURF_stand, 2), "% /", round(TN_SURF_stand, 2), "%")